# Detecção de fraude em seguro de veículo



## Sobre

### Objetivo

todo

### Base de dados

todo

Disponível em: https://www.kaggle.com/datasets/shivamb/vehicle-claim-fraud-detection

## PARTE 1: Importar bibliotecas

In [23]:
import pandas as pd
import scipy.stats as stats

## PARTE 2: Importar base de dados


In [2]:
url_dataset = "https://github.com/peuvitor/insurance-fraud-detection/blob/main/dataset/fraud_oracle.csv?raw=true"

df_dataset = pd.read_csv(url_dataset)

## PARTE 3: Entendimento da base de dados

### 3.1. Investigar as colunas existentes

Quantas e quais são as colunas? Quais são os seus respectivos tipos?

In [3]:
df_dataset.head()

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,...,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy
0,Dec,5,Wednesday,Honda,Urban,Tuesday,Jan,1,Female,Single,...,3 years,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability
1,Jan,3,Wednesday,Honda,Urban,Monday,Jan,4,Male,Single,...,6 years,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision
2,Oct,5,Friday,Honda,Urban,Thursday,Nov,2,Male,Married,...,7 years,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision
3,Jun,2,Saturday,Toyota,Rural,Friday,Jul,1,Male,Married,...,more than 7,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994,Liability
4,Jan,5,Monday,Honda,Urban,Tuesday,Feb,2,Female,Single,...,5 years,31 to 35,No,No,External,none,no change,1 vehicle,1994,Collision


In [4]:
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15420 entries, 0 to 15419
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Month                 15420 non-null  object
 1   WeekOfMonth           15420 non-null  int64 
 2   DayOfWeek             15420 non-null  object
 3   Make                  15420 non-null  object
 4   AccidentArea          15420 non-null  object
 5   DayOfWeekClaimed      15420 non-null  object
 6   MonthClaimed          15420 non-null  object
 7   WeekOfMonthClaimed    15420 non-null  int64 
 8   Sex                   15420 non-null  object
 9   MaritalStatus         15420 non-null  object
 10  Age                   15420 non-null  int64 
 11  Fault                 15420 non-null  object
 12  PolicyType            15420 non-null  object
 13  VehicleCategory       15420 non-null  object
 14  VehiclePrice          15420 non-null  object
 15  FraudFound_P          15420 non-null

### 3.2. Valores únicos de cada coluna

Verificar quais os possíveis valores encontrados na base de dados para cada atributo. Além disso, essa parte será útil para identificar possíveis valores faltantes ou incomuns.

In [5]:
for coluna in df_dataset.columns:
    valores_unicos = df_dataset[coluna].unique()
    print(f"--> '{coluna}' possui {len(valores_unicos)} valores únicos, são eles:\n {valores_unicos}")

--> 'Month' possui 12 valores únicos, são eles:
 ['Dec' 'Jan' 'Oct' 'Jun' 'Feb' 'Nov' 'Apr' 'Mar' 'Aug' 'Jul' 'May' 'Sep']
--> 'WeekOfMonth' possui 5 valores únicos, são eles:
 [5 3 2 4 1]
--> 'DayOfWeek' possui 7 valores únicos, são eles:
 ['Wednesday' 'Friday' 'Saturday' 'Monday' 'Tuesday' 'Sunday' 'Thursday']
--> 'Make' possui 19 valores únicos, são eles:
 ['Honda' 'Toyota' 'Ford' 'Mazda' 'Chevrolet' 'Pontiac' 'Accura' 'Dodge'
 'Mercury' 'Jaguar' 'Nisson' 'VW' 'Saab' 'Saturn' 'Porche' 'BMW' 'Mecedes'
 'Ferrari' 'Lexus']
--> 'AccidentArea' possui 2 valores únicos, são eles:
 ['Urban' 'Rural']
--> 'DayOfWeekClaimed' possui 8 valores únicos, são eles:
 ['Tuesday' 'Monday' 'Thursday' 'Friday' 'Wednesday' 'Saturday' 'Sunday'
 '0']
--> 'MonthClaimed' possui 13 valores únicos, são eles:
 ['Jan' 'Nov' 'Jul' 'Feb' 'Mar' 'Dec' 'Apr' 'Aug' 'May' 'Jun' 'Sep' 'Oct'
 '0']
--> 'WeekOfMonthClaimed' possui 5 valores únicos, são eles:
 [1 4 2 3 5]
--> 'Sex' possui 2 valores únicos, são eles:
 ['Femal

### 3.3. Resumo sobre as colunas

Breve entendimento de cada atributo. Levando em conta o contexto do problema, em geral o nome da coluna é autoexplicativo, porém, em alguns momentos pode dar margem à mais de uma interpretação. Aqui explicarei o que é cada atributo, como trabalharei com ele e problemas que precisam ser investigados.

1. **Month** (object):  
- considerarei como o mês em que o acidente ocorreu
- mês abreviado (3 primeiras letras, em inglês)
- contém os 12 possíveis meses, não há valores faltantes nem incompatíveis

2. **WeekOfMonth** (int64):
- considerarei como a semana do mês em que o acidente ocorreu
- contém valores inteiros de 1 a 5

3. **DayOfWeek** (object):
- considerarei como o dia da semana em que o acidente ocorreu
- nome do dia da semana completo, em inglês
- contém os 7 possíveis dias da semana, não há valores faltantes nem incompatíveis

4. **Make** (object):
- considerarei como a marca do veículo envolvido no acidente
- contém uma lista com 19 fabricantes

5. **AccidentArea** (object):
- informa se o acidente ocorreu em uma área rural ou urbana

6. **DayOfWeekClaimed** (object):
- dia da semana referente ao pedido de seguro
- nome do dia da semana completo, em inglês
- contém 8 valores únicos (7 possíveis dias da semana + '0'), a seguir serão investigados mais a fundo

7. **MonthClaimed** (object):
- mês referente ao pedido de seguro
- mês abreviado (3 primeiras letras, em inglês)
- contém 13 valores únicos (12 possíves meses + '0'), a seguir serão investigados mais a fundo

8. **WeekOfMonthClaimed** (int64):
- semana do mês referente ao pedido de seguro
- contém valores inteiros de 1 a 5

9. **Sex** (object):
- é o sexo biológico da pessoa envolvida no acidente ou da pessoa solicitando o seguro? (considerarei a primeira opção)
- pode ser: sexo masculino ou feminino

10. **MaritalStatus** (object):
- é o estado civil da pessoa envolvida no acidente ou da pessoa solicitando o seguro? (considerarei a primeira opção)
- pode ser: solteiro, casado, divorciado ou viúvo

11. **Age** (int64):
- é a idade da pessoa envolvida no acidente ou da pessoa solicitando o seguro? (considerarei a primeira opção)
- valores inteiros de 16 a 80 (a seguir, o valor '0' será investigado mais a fundo)

12. **Fault** (object):
- indica quem foi considerado como o responsável pelo acidente
- pode ser: titular do seguro ou terceiro

13. **PolicyType** (object):
- indica o tipo da apólice contratada
- contém uma lista com 9 tipos, onde cada um é composto por 'categoria do veículo' + 'tipo do seguro'
    - categoria do veículo: sport, sedan, utility
    - tipo de seguro: liability, all perils, collision
- já existe um atributo para indicar a categoria do veículo (VehicleCategory) e outra para indicar o tipo de seguro (BasePolicy), a seguir verificarei como tratar essas informações duplicadas

14. **VehicleCategory** (object):
- indica a categoria do veículo registrado na apólice/envolvido no acidente
- pode ser: sport, sedan, utility

15. **VehiclePrice** (object):
- indica o preço do veículo envolvido no acidente
- contém 6 faixas de valores (provavelmente em dólares) 
- duas abordagens serão avaliadas: considerar como um dado categórico ou substituir os intervalos pelo valor médio

16. **FraudFound_P** (int64):
- indica se aquela solicitação foi ou não uma fraude
- é o atributo alvo

17. **PolicyNumber** (int64):
- identifica cada pedido de seguro
- contém 15420 valores inteiros únicos, de 1 a 15420
- a seguir será verificada a relação entre o número da linha e o PolicyNumber, possivelmente são a mesma coisa

18. **RepNumber** (int64):
- indica o número do representante
- contém valores inteiros de 1 a 16
- não fica claro a relevância do parâmetro para o problema, a seguir isso será investigado

19. **Deductible** (int64):
- indica o valor da franquia do seguro
- 4 opções de valores inteiros: 300, 400, 500 e 700

20. **DriverRating** (int64):
- alguma métrica de avaliação do motorista
- contém valores inteiros de 1 a 4
- não fica claro se esses dados se traduzem em categorias ou se estão dentro de um certo intervalo de avaliação

21. **Days_Policy_Accident** (object):
- será considerado como o número de dias que se passaram entre a compra da apólice e o acidente
- contém 5 faixas de valores 
- duas abordagens serão avaliadas: considerar como um dado categórico ou substituir os intervalos pelo valor médio

22. **Days_Policy_Claim** (object):
- será considerado como o número de dias que se passaram entre a compra da apólice e o pedido de indenização
- contém 4 faixas de valores 
- duas abordagens serão avaliadas: considerar como um dado categórico ou substituir os intervalos pelo valor médio

23. **PastNumberOfClaims** (object):
- indica quantas vezes a mesma pessoa realizou pedidos de indenização
- contém 4 faixas de valores

24. **AgeOfVehicle** (object):
- indica a idade do veículo (considerarei referenre ao momento do acidente)
- contém 8 faixas de valores 
- duas abordagens serão avaliadas: considerar como um dado categórico ou substituir os intervalos pelo valor médio

25. **AgeOfPolicyHolder** (object):
- indica a idade do titular da apólice
- contém 9 faixas de valores
- duas abordagens serão avaliadas: considerar como um dado categórico ou substituir os intervalos pelo valor médio

26. **PoliceReportFiled** (object):
- indica se foi feito um boletim de ocorrência para o acidente
- pode ser: sim ou não

27. **WitnessPresent** (object):
- indica se uma testemunha estava presente no momento do acidente
- pode ser: sim ou não

28. **AgentType** (object):
- classifica um agente (relacionado ao acidente ou ao pedido) como externo ou interno
- não fica claro a relevância do parâmetro para o problema, a seguir isso será investigado

29. **NumberOfSuppliments** (object):
- indica algum tipo de suplemento no seguro
- contém 4 faixas de valores
- duas abordagens serão avaliadas: considerar como um dado categórico ou substituir os intervalos pelo valor médio
- não fica claro a relevância do parâmetro para o problema, a seguir isso será investigado

30. **AddressChange_Claim** (object):
- indica quantos anos se passaram entre a última vez que foi registrado uma mudança de endereço e o pedido de indenização
- contém 5 faixas de valores
- duas abordagens serão avaliadas: considerar como um dado categórico ou substituir os intervalos pelo valor médio

31. **NumberOfCars** (object):
- considerarei como o número de carros cobertos pela apólice (outra possibilidade seria o número de carros envolvidos no acidente)
- contém 5 faixas de valores
- duas abordagens serão avaliadas: considerar como um dado categórico ou substituir os intervalos pelo valor médio

32. **Year** (int64):
- considerarei como o ano em que o acidente ocorreu
- pode ser: 1994, 1995 e 1996

33. **BasePolicy** (object):
- indica o tipo de seguro contratado
- pode ser: liability, all perils, collision

## PARTE 4: Tratamento dos dados

### 4.1. Verificar relação entre os índices das linhas do dataset e 'PolicyNumber'

PolicyNumber é igual ao índice da linha+1?

In [6]:
df_dataset['PolicyNumber'].describe

<bound method NDFrame.describe of 0            1
1            2
2            3
3            4
4            5
         ...  
15415    15416
15416    15417
15417    15418
15418    15419
15419    15420
Name: PolicyNumber, Length: 15420, dtype: int64>

In [7]:
df_dataset_total_linhas = len(df_dataset)

PolicyNumber_e_indice = sum(1 if df_dataset['PolicyNumber'].iloc[indice] == indice+1 else 0 \
                                    for indice in range(df_dataset_total_linhas))

print(f"Número de linhas do dataset: {df_dataset_total_linhas}")
print(f"Quantidade de linhas onde PolicyNumber = indice+1: {PolicyNumber_e_indice}")

Número de linhas do dataset: 15420
Quantidade de linhas onde PolicyNumber = indice+1: 15420


Como a suposição foi provada verdadeira, esta coluna será ignorada, pois não apresenta utilidade para o problema em questão.

In [8]:
df_dataset.drop(columns='PolicyNumber', inplace=True)

### 4.2. Tratar o valor 0 encontrado em 'DayOfWeekClaimed', 'MonthClaimed' e 'Age'

In [9]:
##### 'DayOfWeekClaimed' e 'MonthClaimed'

DayOfWeekClaimed_zeros = df_dataset.index[df_dataset['DayOfWeekClaimed'] == '0'].tolist()
MonthClaimed_zeros = df_dataset.index[df_dataset['MonthClaimed'] == '0'].tolist()

print("DayOfWeekClaimed e MonthClaimed possuem o valor '0' na(s) mesma(s) linha(s):", DayOfWeekClaimed_zeros == MonthClaimed_zeros)
print("Índice(s):", DayOfWeekClaimed_zeros)

# como trata-se apenas de uma única ocorrência, a linha correspondente será eliminada
df_dataset = df_dataset.drop(DayOfWeekClaimed_zeros)
df_dataset.reset_index(drop=True, inplace=True)


DayOfWeekClaimed e MonthClaimed possuem o valor '0' na(s) mesma(s) linha(s): True
Índice(s): [1516]


In [10]:
##### 'Age'

Age_zeros = df_dataset.index[df_dataset['Age'] == 0].tolist()
print(f"Age possui o valor '0' em {len(Age_zeros)} linhas. \n")

# partirei do seguinte questionamento: nessas ocasiões, existe alguma relação com outra coluna que pode ser útil?

for coluna in df_dataset.columns:
    linhas_age0 = df_dataset.loc[df_dataset['Age']==0, coluna].unique()
    if len(linhas_age0) < 2 and coluna != 'Age':
        print(f"--> Correspondência para '{coluna}': {linhas_age0}")


Age possui o valor '0' em 319 linhas. 

--> Correspondência para 'Sex': ['Male']
--> Correspondência para 'MaritalStatus': ['Single']
--> Correspondência para 'Days_Policy_Accident': ['more than 30']
--> Correspondência para 'Days_Policy_Claim': ['more than 30']
--> Correspondência para 'AgeOfPolicyHolder': ['16 to 17']


In [11]:
df_dataset['AgeOfPolicyHolder'].value_counts().sort_index()

16 to 17     319
18 to 20      15
21 to 25     108
26 to 30     613
31 to 35    5593
36 to 40    4043
41 to 50    2828
51 to 65    1392
over 65      508
Name: AgeOfPolicyHolder, dtype: int64

Verificando as colunas que possuem sempre a mesma correspondência, à primeira vista, aquela que pode dar uma solução para Age==0 é 'AgeOfPolicyHolder'.

E, de fato, considerando todo o dataset, temos uma frequência de 319 para '16 to 17' em AgeOfPolicyHolder. Exatamente a mesma quantidade de linhas com Age==0.

Investigarei o seguinte: 'Age' sempre está dentro do intervalo de 'AgeOfPolicyHolder'?

In [12]:
# extrair os limites dos intervalos em 'AgeOfPolicyHolder'
get_intervalo_AgeOfPolicyHolder = lambda linha: [int(linha[:2]), int(linha[-2:])] if 'to' in linha else [int(linha[-2:]), 200]

df_dataset_total_linhas = len(df_dataset)
total = 0

for indice in range(df_dataset_total_linhas):
    idade = df_dataset['Age'].iloc[indice]
    intervalo_idade = get_intervalo_AgeOfPolicyHolder(df_dataset['AgeOfPolicyHolder'].iloc[indice])
    if ~(idade >= intervalo_idade[0] and idade <= intervalo_idade[1]): total += 1

print(f"Em {total} linhas 'Age' não corresponde a 'AgeOfPolicyHolder'. O que representa {total/df_dataset_total_linhas:.2%} do total de linhas.")

Em 7241 linhas 'Age' não corresponde a 'AgeOfPolicyHolder'. O que representa 46.96% do total de linhas.


A resposta para o questionamento é 'não'. Ou seja, o indivíduo envolvido no acidente não necessariamente é a pessoa titular da apólice do seguro. 

Apesar disso, para não excluir essa quantidade de linhas do dataset, a opção escolhida para dar seguimento ao projeto é a de substituir o valor de 0 de 'Age' pela média do intervalo de 'AgeOfPolicyHolder'.

In [13]:
df_dataset.loc[df_dataset['Age']==0, 'Age'] = 16.5

Age_zeros = df_dataset.index[df_dataset['Age'] == 0].tolist()
print(f"Age possui o valor '0' em {len(Age_zeros)} linhas. \n")

Age_mean = df_dataset.index[df_dataset['Age'] == 16.5].tolist()
print(f"Age possui o valor '16.5' em {len(Age_mean)} linhas. \n")

Age possui o valor '0' em 0 linhas. 

Age possui o valor '16.5' em 319 linhas. 



### 4.3. Verificar relação entre 'PolicyType', 'VehicleCategory' e 'BasePolicy'

PolicyType = "VehicleCategory - BasePolicy"?

In [14]:
df_dataset[['PolicyType', 'VehicleCategory', 'BasePolicy']].describe

<bound method NDFrame.describe of                PolicyType VehicleCategory  BasePolicy
0       Sport - Liability           Sport   Liability
1       Sport - Collision           Sport   Collision
2       Sport - Collision           Sport   Collision
3       Sedan - Liability           Sport   Liability
4       Sport - Collision           Sport   Collision
...                   ...             ...         ...
15414   Sedan - Collision           Sedan   Collision
15415   Sedan - Liability           Sport   Liability
15416   Sedan - Collision           Sedan   Collision
15417  Sedan - All Perils           Sedan  All Perils
15418   Sedan - Collision           Sedan   Collision

[15419 rows x 3 columns]>

In [15]:
df_dataset_total_linhas = len(df_dataset)
total = 0

for indice in range(df_dataset_total_linhas):
    PolicyType = df_dataset['PolicyType'].iloc[indice]
    VehicleCategory_BasePolicy = f"{df_dataset['VehicleCategory'].iloc[indice]} - {df_dataset['BasePolicy'].iloc[indice]}"
    if PolicyType == VehicleCategory_BasePolicy: total += 1

print(f"'PolicyType' corresponde a 'VehicleCategory - BasePolicy' em {total} linhas. O que representa {total/df_dataset_total_linhas:.2%} do total de linhas.")

'PolicyType' corresponde a 'VehicleCategory - BasePolicy' em 10432 linhas. O que representa 67.66% do total de linhas.


A suposição foi provada falsa, portanto, nada será feito com essas três colunas. Mesmo se a suposição fosse verdadeira, ter as informações tanto separadas quanto combinadas pode ser útil para o modelo a ser construído.

## PARTE 5: Investigar relações com a variável alvo

Usar de testes estatísticos e recursos gráficos para verificar as relações entre os atributos e o target ('FraudFound_P').

### 5.1 Teste de Independência

Considerando os diferentes tipos de atributos (numéricos, binários, categóricos), a abordagem escolhida leva em conta a tabela de contigência para cada par coluna e target. Será realizado o teste de independência chi-quadrado, onde a hipótese nula **$H_0$** é que as colunas **não têm relação** e a hipótese alternativa **$H_1$** é que **existe relação** entre as colunas. O nível de significância (alfa) considerado é de 0,05.

Para os resultados dos testes:

- Se o valor-p encontrado for **maior que alfa**, não rejeitamos a hipótese nula. Podemos dizer que o resultado do teste não detecta uma relação significativa entre as variáveis;

- Se o valor-p encontrado for **menor que alfa**, podemos dizer que há evidências estatísticas suficientes de que existe alguma relação entre as variáveis.

In [50]:
# exemplo de tabela de contigência
pd.crosstab(df_dataset['MaritalStatus'], df_dataset['FraudFound_P'])

FraudFound_P,0,1
MaritalStatus,,
Divorced,73,3
Married,9986,639
Single,4405,278
Widow,32,3


In [56]:
# teste de independêndia chi-quadrado
nivel_significancia = 0.05
teste_independencia = []

for coluna in df_dataset.columns:
    tabela_contigencia = pd.crosstab(df_dataset[coluna], df_dataset['FraudFound_P'])
    chi2_val, valor_p, _, __ = stats.chi2_contingency(tabela_contigencia)
    if valor_p < nivel_significancia:
        teste_independencia.append([coluna, chi2_val, valor_p])

print("Há evidências estatísticas suficientes de que existe alguma relação entre FraudFound_P e os seguintes atributos:")

df_teste_independencia = pd.DataFrame(teste_independencia, columns=['Atributo', '\u03C7\u00B2', 'Valor-p'])
df_teste_independencia.sort_values(by='Valor-p', inplace=True)
print(df_teste_independencia)

Há evidências estatísticas suficientes de que existe alguma relação entre FraudFound_P e os seguintes atributos:
                Atributo            χ²       Valor-p
10          FraudFound_P  15401.236138  0.000000e+00
7             PolicyType    437.491381  1.768441e-89
20            BasePolicy    402.947238  3.170436e-88
8        VehicleCategory    290.980893  6.520817e-64
6                  Fault    264.984556  1.406180e-59
18   AddressChange_Claim    104.722693  9.704718e-22
11            Deductible     72.406255  1.302831e-15
9           VehiclePrice     67.836116  2.888324e-13
13    PastNumberOfClaims     53.541755  1.405198e-11
1                   Make     59.815292  2.191573e-06
3           MonthClaimed     42.200514  1.495245e-05
2           AccidentArea     16.901858  3.936304e-05
15     AgeOfPolicyHolder     33.104861  5.896560e-05
4                    Sex     13.495678  2.391135e-04
17   NumberOfSuppliments     18.155527  4.085276e-04
5                    Age    109.664968 

### 5.2 

Apenas com os atributos que mostram ter uma potencial relação com a variável alvo, utilizarei de recursos gráficos para visualizar mais a fundo essas relações e ver se é possível extrair algum insight.

In [57]:
# todo

## PARTE 6: Encoding the data

o que é RepNumber? o que é DriverRating? o que é AgentType? o que é NumberOfSuppliments?
diferença entre mês do acidente e mês de solicitação (pode citar isso, mas a coluna Days_Policy_Claim e/ou Days_Policy_Accident já deve ajudar)

dados categóricos (binário, one hot encoder)

VehiclePrice, Days_Policy_Accident, Days_Policy_Claim, AgeOfVehicle, AgeOfPolicyHolder, NumberOfSuppliments, AddressChange_Claim, NumberOfCars: substituir pela média ou categorizar?

AccidentArea: 1=Urban, 0=Rural
Sex: 1=Female, 0=Male
Fault: 1=Policy Holder, 0=Third Party
PoliceReportFiled: 1=Yes, 0=No
WitnessPresent: 1=Yes, 0=No
AgentType: 1=External 0=Internal